In [ ]:
from diffusers import DDIMScheduler
from diffusers import StableDiffusionPipeline
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 将 model_path 设置为 Stable-Diffusion-v1.5 的模型路径或名字
model_or_name = "/root/autodl-tmp/model/"

ip_adaptor_path = "/root/autodl-tmp/ip-composition-adapter/sd15_models"

scheduler = DDIMScheduler.from_pretrained(model_or_name, subfolder="scheduler")
model = StableDiffusionPipeline.from_pretrained(
    model_or_name, scheduler=scheduler, torch_dtype=torch.float
)

model.load_ip_adapter(ip_adaptor_path, subfolder=".", weight_name="ip_plus_composition_sd15.safetensors", 
                      image_encoder_folder="image_encoder", device=device)

# 设置随机种子
seed = 4
generator = torch.Generator(device=device)
generator.manual_seed(seed)

model.generator = generator

In [ ]:
# 设置一些路径和参数
from inversion_free import gen_inversion_free
from utils import img2latent, latent2img
from datetime import datetime
from PIL import Image
import os

# 加载图片
# 第一级目录
img_dir_src_1 = "./samples"
img_dir_tar_1 = img_dir_src_1
# 第二级目录
img_dir_src_2 = "girl"
src_img_dir = os.path.join(img_dir_src_1, img_dir_src_2)
tar_img_dir_2 = "girl5"
tar_img_dir = os.path.join(img_dir_tar_1, tar_img_dir_2)

def get_image_file(path):
    img_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif')
    img_files = [f for f in os.listdir(path) if f.lower().endswith(img_extensions)]
    return img_files[0]

src_img_file = get_image_file(src_img_dir)
tar_img_file = get_image_file(tar_img_dir)

model = model.to("cpu")

src_latent = img2latent(os.path.join(src_img_dir, src_img_file), model)
tar_latent = img2latent(os.path.join(tar_img_dir, tar_img_file), model)

src_image = Image.open(os.path.join(src_img_dir, src_img_file))
tar_image = Image.open(os.path.join(tar_img_dir, tar_img_file))
model = model.to(device)
model.set_ip_adapter_scale(0.6)
pipeline = model
results = pipeline(
    prompt="a girl wearing a hat",
    # image=src_image,
    ip_adapter_image=src_image,
    negative_prompt="worst quality, low quality",
    generator=generator,
    strength=0.6,
).images[0]

results.save("output_img/test2.png")

# 获取嵌入向量
src_embedding_o = model.prepare_ip_adapter_image_embeds(
    ip_adapter_image=src_image,
    ip_adapter_image_embeds=None,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=True,
)

tar_embedding_o = model.prepare_ip_adapter_image_embeds(
    ip_adapter_image=tar_image,
    ip_adapter_image_embeds=None,
    device=device,
    num_images_per_prompt=1,
    do_classifier_free_guidance=True,
)

src_embedding_uc, src_embedding = model.unet.encoder_hid_proj(src_embedding_o)[0][:, 0, :, :].chunk(2)
tar_embedding_uc, tar_embedding = model.unet.encoder_hid_proj(tar_embedding_o)[0][:, 0, :, :].chunk(2)
# model.unload_ip_adapter()

In [ ]:
from utils import load_multitoken_tokenizer
from multi_token_clip import MultiTokenCLIPTokenizer
from transformers.models.clip.modeling_clip import CLIPTextModel

src_placeholders = "<src>"
tar_placeholders = "<tar>"

text_encoder = CLIPTextModel.from_pretrained(model_or_name, subfolder="text_encoder", revision=False)
tokenizer = MultiTokenCLIPTokenizer.from_pretrained(model_or_name, subfolder="tokenizer")

# placeholder_dict = {
#     "<src>": src_embedding[0][:3],
#     "<tar>": tar_embedding[0][:3],
# }
# 
# load_multitoken_tokenizer(tokenizer, text_encoder, placeholder_dict, tar_placeholders)
# load_multitoken_tokenizer(tokenizer, text_encoder, placeholder_dict, src_placeholders)

model.text_encoder = text_encoder
model.tokenizer = tokenizer

In [ ]:
from attention_control import make_controller

# 设置输出目录并生成
date = datetime.now().strftime("%Y-%m-%d")
output_dir = f"./output_img/{str(date)}-{img_dir_src_2}-to-{tar_img_dir_2}/"
os.makedirs(output_dir, exist_ok=True)

time = datetime.now().strftime("%H-%M-%S")
num_inference_steps = 20
save_all = True

for attn in [False]:
    for cfg in [3]:
        # 是否使用注意力控制器
        use_attention = attn
        
        # 选择权重的类型
        inclination = "none-tar"
        mode = "cosine"
        cfg_guidance_scale = cfg
        
        # src 和 tar 方向的系数
        src_coefficient = 0
        tar_coefficient = 0
        
        output_img_name = f"{time}_{src_img_file[:-4]}2{tar_img_file[:-4]}_{num_inference_steps}steps_{inclination}_{mode[:3]}_cfg{cfg_guidance_scale}_src{src_coefficient}_tar{tar_coefficient}_attn{use_attention}"
        
        if use_attention:
            # 设置注意力控制器
            placeholder = [src_placeholders, tar_placeholders]
            cross_injection_ratio = 0.2
            self_injection_ratio = 0.7
            eq_param = {
                'words' : (placeholder[-1],),
                'values' : (0.5,),
            }
            controller = make_controller(
                prompts=placeholder,
                tokenizer=tokenizer,
                is_replace_controller=False,
                cross_replace_steps={'default_': cross_injection_ratio},
                self_replace_steps=self_injection_ratio,
                equilizer_params=eq_param,
            )
            model_2 = None
        else:
            controller = None
            model_2 = None
            
        # 生成图片
        latents = gen_inversion_free(
            model.to(device), src_latent, tar_latent, src_embedding, tar_embedding,
            num_inference_steps=num_inference_steps, mode=mode, inclination=inclination,
            cfg_guidance=cfg_guidance_scale, src_coef=src_coefficient, tar_coef=tar_coefficient,
            controller=controller, return_all=save_all, kwarg={"image_embeds": tar_embedding_o},
        )
        
        
        if save_all:
            for i, latent in enumerate(latents):
                img = latent2img(latent[0].detach(), model)
                print(f"Saving {output_img_name}_{i}.png in {output_dir}")
                img.save(os.path.join(output_dir, f"{output_img_name}_{i}.png"))
        else:
            img = latent2img(latents[-1].detach(), model)
            print(f"Saving {output_img_name}.png in {output_dir}")
            img.save(os.path.join(output_dir, f"{output_img_name}.png"))